In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_sub = pd.read_csv('sample_submission2.csv')

In [4]:
!pip install git+https://github.com/MaxHalford/xam --upgrade

  Cloning https://github.com/MaxHalford/xam to c:\users\hp\appdata\local\temp\pip-req-build-vsus14dn


  Error [WinError 2] The system cannot find the file specified while executing command git clone -q https://github.com/MaxHalford/xam C:\Users\HP\AppData\Local\Temp\pip-req-build-vsus14dn
Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [5]:
# drop ID column
train.drop('EmployeeNo', axis=1, inplace=True)

test_id = test['EmployeeNo']
test.drop('EmployeeNo', axis=1, inplace=True)

In [6]:
cat_cols = list(train.select_dtypes(include='object').columns)
num_cols = list(train.select_dtypes(exclude='object').columns)

In [7]:
train.isna().sum()

Division                                  0
Qualification                          1679
Gender                                    0
Channel_of_Recruitment                    0
Trainings_Attended                        0
Year_of_birth                             0
Last_performance_score                    0
Year_of_recruitment                       0
Targets_met                               0
Previous_Award                            0
Training_score_average                    0
State_Of_Origin                           0
Foreign_schooled                          0
Marital_Status                            0
Past_Disciplinary_Action                  0
Previous_IntraDepartmental_Movement       0
No_of_previous_employers                  0
Promoted_or_Not                           0
dtype: int64

In [8]:
train['Qualification'].value_counts()

First Degree or HND         25578
MSc, MBA and PhD            10469
Non-University Education      586
Name: Qualification, dtype: int64

In [9]:
# train['Missing_quaifications'] =  train['Qualification'].isna().astype('int64')
train['Qualification'].fillna('First Degree or HND', inplace=True)

# test['Missing_quaifications'] =  test['Qualification'].isna().astype('int64')
test['Qualification'].fillna('First Degree or HND', inplace=True)

In [10]:
train[cat_cols].head()

,Division,Qualification,Gender,Channel_of_Recruitment,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers
0,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,ANAMBRA,No,Married,No,No,0
1,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,ANAMBRA,Yes,Married,No,No,0
2,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,KATSINA,Yes,Married,No,No,0
3,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,NIGER,Yes,Single,No,No,1
4,Information and Strategy,First Degree or HND,Male,Direct Internal process,AKWA IBOM,Yes,Married,No,No,1


In [11]:
for col in cat_cols:
    print("{}: {}".format(col, train[col].nunique()))

Division: 9
Qualification: 3
Gender: 2
Channel_of_Recruitment: 3
State_Of_Origin: 37
Foreign_schooled: 2
Marital_Status: 3
Past_Disciplinary_Action: 2
Previous_IntraDepartmental_Movement: 2
No_of_previous_employers: 7


In [12]:
import xam

ModuleNotFoundError: No module named 'xam'

In [13]:
# mean encoding

mean_enc = xam.feature_extraction.BayesianTargetEncoder(columns=cat_cols)

y = train['Promoted_or_Not']
train_enc = train.drop('Promoted_or_Not', axis=1)
mean_enc.fit(train_enc, y)

train_enc = mean_enc.transform(train_enc)
test_enc = mean_enc.transform(test)

NameError: name 'xam' is not defined

In [14]:
train_enc.drop(cat_cols, axis=1, inplace=True)
test_enc.drop(cat_cols, axis=1, inplace=True)

NameError: name 'train_enc' is not defined

In [ ]:
train_enc.head()

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
cat_col_train = train_enc[['Targets_met','Last_performance_score']]
cat_col_test = test_enc[['Targets_met','Last_performance_score']]

In [ ]:
k_train = KMeans(n_clusters=3)
k_train.fit(cat_col_train)
train_enc['Cluster'] = k_train.predict(cat_col_train)

k_test = KMeans(n_clusters=3)
k_test.fit(cat_col_test)
test_enc['Cluster'] = k_test.predict(cat_col_test)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_enc, y, test_size=0.3, random_state=1)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import BaggingClassifier, VotingClassifier

import lightgbm as lgb
# import xgboost as xgb
import catboost as cb

In [ ]:
def train_and_score(model, X, y, test, test_y):
    model.fit(X, y)
    pred = model.predict(test)
    print("F1_score is {}".format(f1_score(pred, test_y)))

In [ ]:
def max_train_and_score(models, X, y, test, test_y, full=False):
    pred_array = pd.DataFrame()
    
    for i,model in enumerate(models):
        model.fit(X, y)
        pred = model.predict(test)
        pred_array['model{}'.format(i)] = pred
        max_pred = round(pred_array.mean(axis=1)).astype('int64')
        
    if full:
        pass
    else:
        print("F1_score is {}".format(f1_score(max_pred, test_y)))
    
    return max_pred

In [ ]:
catb = cb.CatBoostClassifier(iterations=1000, use_best_model=True, eval_metric='F1', random_seed=1)
catb.fit(X_train,y_train, eval_set=(X_test, y_test))

In [ ]:
cb_model = cb.CatBoostClassifier(iterations=700, random_seed=1, silent=True)
train_and_score(cb_model, X_train, y_train, X_test, y_test)

In [ ]:
best_lgb = lgb.LGBMClassifier(n_estimators=1200, max_depth=4, random_state = 1)
train_and_score(best_lgb, X_train, y_train, X_test, y_test)

In [ ]:
cb_model.score(X_train,y_train)

In [ ]:
best_lgb.score(X_test,y_test)

In [ ]:
sca = StandardScaler()

sca.fit(train_enc)
train = sca.transform(train_enc)
test = sca.transform(test)

cb_model.fit(train, y)
final_pred = cb_model.predict(test)

In [ ]:
sample_sub['EmployeeNo'] = test_id
sample_sub['Promoted_or_Not'] = final_pred.astype('int64')
sample_sub.head()

In [ ]:
sample_sub['Promoted_or_Not'].value_counts()

In [ ]:
sample_sub.to_csv('cb_508.csv', index=False)